In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)


In [4]:
# Load the custom dataset
with open('generated_2_jokes.txt', 'r', encoding='utf-8') as f:
    dataset = f.read().split('\n')

# Tokenize the dataset
inputs = tokenizer(dataset, return_tensors='pt', max_length=512, truncation=True, padding=True)

# Create training dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone().detach()  # Use input_ids as labels
        return item

train_dataset = CustomDataset(inputs)


In [5]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=1,               # number of training epochs
    per_device_train_batch_size=2,    # batch size for training
    warmup_steps=10,                  # number of warmup steps
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    report_to='none'                  # Disable wandb logging
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()


<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.380700
1000,0.246000
1500,0.230300
2000,0.223900
2500,0.223200
3000,0.209600
3500,0.201500


<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a 

Step,Training Loss
500,0.380700
1000,0.246000
1500,0.230300
2000,0.223900
2500,0.223200
3000,0.209600
3500,0.201500
4000,0.205000
4500,0.195300


<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-4-3f2dd2a8a8d6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=4995, training_loss=0.2328984456258016, metrics={'train_runtime': 677.864, 'train_samples_per_second': 14.737, 'train_steps_per_second': 7.369, 'total_flos': 514923920640000.0, 'train_loss': 0.2328984456258016, 'epoch': 1.0})

In [7]:
# Function to generate text
def generate_text(prompt, max_length=50):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    attention_mask = (inputs != tokenizer.pad_token_id).long().to('cuda')
    outputs = model.generate(inputs, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Once upon a time"
generated_text = generate_text(prompt)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Once upon a time when I was a kid, I was a little bit of a pessimist. I was always thinking, "I'm going to be a little bit of a pessimist."
